## Запрос к API из кода
Давайте сделаем запрос из кода. Будем пользоваться всё той же библиотекой requests.

In [2]:
import requests  
  
url = 'https://api.vk.com/method/users.get'  
token = '5d2f1be85d2f1be85d2f1be89d5d5901ae55d2f5d2f1be83d21d4ebe69afdbbac49afae'
params = {'user_id': 1, 'v': 5.95, 'fields': 'sex,bdate', 'access_token': token, 'lang': 'ru'}  
  
# Мы можем выставить параметры запроса через аргумент params  
response = requests.get(url, params=params)  
response.text  

'{"response":[{"first_name":"Павел","id":1,"last_name":"Дуров","can_access_closed":true,"is_closed":false,"sex":2,"bdate":"10.10.1984"}]}'

Мы получили строку в формате JSON. Как мы помним по первому разделу, её можно преобразовать в словарь методом json и после этого обращаться к различным полям. Кроме того, такие большие вложенные словари нагляднее выводить с помощью функции pprint (~pretty print, красивый вывод), которым мы и воспользуемся

In [3]:
from pprint import pprint  
  
pprint(response.json())  

{'response': [{'bdate': '10.10.1984',
               'can_access_closed': True,
               'first_name': 'Павел',
               'id': 1,
               'is_closed': False,
               'last_name': 'Дуров',
               'sex': 2}]}


In [4]:
user = response.json()['response'][0]  
  
# Выведем дату рождения  
print(user['bdate'])  
  
# Выведем имя  
print(user['first_name'])  

10.10.1984
Павел


Данный метод позволяет запрашивать сразу много пользователей (до 1000).  Для этого нужно использовать параметр user_ids и передавать id через запятую в строковом формате, например: '1,2,3'.

In [5]:
ids = ",".join(map(str, range(1, 4)))  
print(ids)  

1,2,3


In [6]:
params = {'user_ids': ids, 'v': 5.95, 'fields': 'bday', 'access_token': token, 'lang': 'ru'}  

pprint(requests.get(url, params=params).json()) 

{'response': [{'can_access_closed': True,
               'first_name': 'Павел',
               'id': 1,
               'is_closed': False,
               'last_name': 'Дуров'},
              {'can_access_closed': False,
               'first_name': 'Александра',
               'id': 2,
               'is_closed': True,
               'last_name': 'Владимирова'},
              {'deactivated': 'deleted',
               'first_name': 'DELETED',
               'id': 3,
               'last_name': ''}]}


### Задание
Используя API, определите долю женщин(sex=1) среди пользователей с id от 1 до 500. Иногда вам будут попадать пользователи, у которых пол не указан (sex=0), их не нужно учитывать в общем числе.

В ответе укажите число, округлив до двух знаков после запятой, например, 0.55

Пример: если у нас будет 300 пользователей sex=1, 100 пользователей с sex=2 и 100 пользователей с sex=0, то в ответе должно быть 0.75

In [7]:
import requests  
from pprint import pprint

In [8]:
url = 'https://api.vk.com/method/users.get' 
token = '5d2f1be85d2f1be85d2f1be89d5d5901ae55d2f5d2f1be83d21d4ebe69afdbbac49afae'
ids = ",".join(map(str, range(1, 501)))  
params = {'user_ids': ids, 'v': 5.95, 'fields': 'sex', 'access_token': token, 'lang': 'ru'} 

response = requests.get(url, params=params)   
users = requests.get(url, params=params).json()['response']

In [9]:
m=w=0
for user in users:
    if user['sex'] == 1:
        w += 1
    elif user['sex'] == 2:
        m += 1
    else:
        continue

print(round(w / (m + w), 2))

0.48


## Ограничения API

#### Ограничения API
В прошлом блоке мы собрали информацию о небольшом наборе пользователей. Теперь перейдем к более реальной задаче — сбору данных о пользователях группы ВКонтакте. 

Стоит отметить, что есть много сервисов, которые выгружают похожую статистику из соцсетей. Однако им свойственны недостатки универсальных решений:

- не могут учитывать всех особенностей вашего проекта;
- считают фиксированный набор метрик, дополнительную обработку данных приходится делать вам;
- не всегда бесплатны и вряд ли позволят работать с большими объемами данных.

#### В этом разделе мы научимся считать произвольные метрики групп, собирая данные из API. 

В текущем шаге мы научимся работать с двумя ограничениями, которые свойственны практически всем системам:

- ограничение на количество вызовов в единицу времени;
- ограничение на количество выгружаемых строк за один запрос.

Ограничение на количество запросов в секунду сделано для того, чтобы избежать чрезмерной нагрузки на серверы системы. В ряде случаев небольшое количество отчетов можно выгрузить, уложившись в этот лимит (например, как мы получили информацию о 500 пользователях в прошлом упражнении). Однако второе ограничение не удастся «обойти» в случае выгрузки больших отчетов. Например, чтобы получить список всех пользователей очень популярной группы, серверу, возможно, придется отправить вам разом лист из миллионов записей.

Давайте рассмотрим, как работать с этими ограничениями на примере выгрузки списка пользователей группы https://vk.com/vk.

Посмотрим в документации, какие методы нам доступны для групп. Для получения списка пользователей группы видим метод groups.getMembers.

Согласно документации, обязательным параметром является ID или короткое имя группы. В нашем случае это vk: https://vk.com/vk. Тестируем, как работает метод в самом простом случае:

In [10]:
import requests  

In [11]:
url = 'https://api.vk.com/method/groups.getMembers'  
token = '5d2f1be85d2f1be85d2f1be89d5d5901ae55d2f5d2f1be83d21d4ebe69afdbbac49afae'
params = {'group_id': 'vk', 'v': 5.95, 'access_token': token}  

response = requests.get(url, params = params)  
data = response.json()  
print(data)  

{'response': {'count': 11456995, 'items': [5, 6, 10, 19, 34, 47, 54, 79, 177, 193, 198, 205, 219, 239, 243, 254, 345, 404, 406, 407, 467, 485, 510, 550, 619, 628, 640, 643, 690, 702, 720, 721, 724, 744, 804, 809, 831, 832, 834, 847, 900, 905, 907, 914, 943, 952, 958, 966, 976, 979, 1000, 1018, 1023, 1032, 1033, 1038, 1039, 1059, 1097, 1131, 1139, 1140, 1159, 1174, 1188, 1290, 1301, 1333, 1334, 1336, 1351, 1359, 1386, 1388, 1406, 1411, 1418, 1432, 1494, 1500, 1531, 1543, 1568, 1586, 1590, 1593, 1598, 1610, 1615, 1632, 1634, 1639, 1650, 1679, 1690, 1697, 1698, 1699, 1700, 1721, 1740, 1754, 1796, 1814, 1820, 1829, 1834, 1839, 1840, 1843, 1858, 1863, 1868, 1869, 1889, 1917, 1943, 1947, 1955, 1969, 2019, 2028, 2050, 2051, 2052, 2059, 2077, 2103, 2145, 2150, 2195, 2201, 2202, 2230, 2236, 2273, 2281, 2294, 2296, 2298, 2376, 2389, 2395, 2403, 2412, 2436, 2456, 2466, 2470, 2484, 2515, 2527, 2539, 2571, 2576, 2592, 2601, 2622, 2644, 2654, 2692, 2706, 2745, 2755, 2767, 2787, 2797, 2827, 2858, 289

In [12]:
len(data['response']['items'])  

1000

#### Получаем всех пользователей
Мы видим, что всего пользователей в группе больше 11 000 000, а получили мы только первую тысячу пользователей группы. Судя по описанию параметра count в документации, это максимум, который может отдать API за один раз.

Для получения следующей тысячи пользователей воспользуемся параметрами count и offset: будем в цикле выгружать по 1000 пользователей (count будет всегда равен 1000), а в каждом следующем шаге цикла увеличим смещение offset на величину count.

Сначала напишем цикл выгрузки первых 20 пользователей со значением count = 5. Т.е. цикл будет выгружать за 1 запрос по 5 пользователей, пока не выгрузит первые 20. Это позволит нам проверить корректность работы цикла перед тем, как делать большие и долгие выгрузки.

Давайте выведем на экран первые 20 пользователей из нашей первой попытки с 1000 пользователей, чтобы было с чем сверить результат выгрузки из 20 пользователей:

In [13]:
users_for_checking = data['response']['items'][:20]  
print(users_for_checking)  

[5, 6, 10, 19, 34, 47, 54, 79, 177, 193, 198, 205, 219, 239, 243, 254, 345, 404, 406, 407]


Теперь используем count и offset, чтобы получить те же id по 5 за раз

In [14]:
count = 5  
offset = 0  
user_ids = []
max_count = 20  
while offset < max_count:  
    print('Выгружаю {} пользователей с offset = {}'.format(count, offset))     
    params = {'group_id': 'vk', 'v': 5.95, 'count': count, 
              'offset': offset, 'access_token': token}     
    
    # такой же запрос как в прошлый раз  
    r = requests.get(url, params = params)  
    data = r.json()     
    user_ids += data['response']['items']   
  
    # увеличиваем смещение на количество строк выгрузки  
    offset += count  

Выгружаю 5 пользователей с offset = 0
Выгружаю 5 пользователей с offset = 5
Выгружаю 5 пользователей с offset = 10
Выгружаю 5 пользователей с offset = 15


In [15]:
print(user_ids)  

[5, 6, 10, 19, 34, 47, 54, 79, 177, 193, 198, 205, 219, 239, 243, 254, 345, 404, 406, 407]


In [16]:
user_ids == users_for_checking  

True

Видим, что подход корректно работает. Теперь мы можем получить всех пользователей, выставив count = 1000 и max_count = data['response']['count'].

### Задание
Используя API, определите id стотысячного пользователя в группе vk . Т.е. если бы у нас не было ограничения на размер запроса, то мы бы запросили data['response']['items'][99999] .

In [20]:
count = 1000  
offset = 0  
user_ids = []
max_count = 101000  
while offset < max_count:  
    print('Выгружаю {} пользователей с offset = {}'.format(count, offset))     
    params = {'group_id': 'vk', 'v': 5.95, 'count': count, 
              'offset': offset, 'access_token': token}     
    
    # такой же запрос как в прошлый раз  
    r = requests.get(url, params = params)  
    data = r.json()     
    user_ids += data['response']['items']   
  
    # увеличиваем смещение на количество строк выгрузки  
    offset += count

Выгружаю 1000 пользователей с offset = 0
Выгружаю 1000 пользователей с offset = 1000
Выгружаю 1000 пользователей с offset = 2000
Выгружаю 1000 пользователей с offset = 3000
Выгружаю 1000 пользователей с offset = 4000
Выгружаю 1000 пользователей с offset = 5000
Выгружаю 1000 пользователей с offset = 6000
Выгружаю 1000 пользователей с offset = 7000
Выгружаю 1000 пользователей с offset = 8000
Выгружаю 1000 пользователей с offset = 9000
Выгружаю 1000 пользователей с offset = 10000
Выгружаю 1000 пользователей с offset = 11000
Выгружаю 1000 пользователей с offset = 12000
Выгружаю 1000 пользователей с offset = 13000
Выгружаю 1000 пользователей с offset = 14000
Выгружаю 1000 пользователей с offset = 15000
Выгружаю 1000 пользователей с offset = 16000
Выгружаю 1000 пользователей с offset = 17000
Выгружаю 1000 пользователей с offset = 18000
Выгружаю 1000 пользователей с offset = 19000
Выгружаю 1000 пользователей с offset = 20000
Выгружаю 1000 пользователей с offset = 21000
Выгружаю 1000 пользоват

In [21]:
print(user_ids[-1])  

6306205


## Ограничение по частоте запросов
В API частот добавляют ограничение по частоте запросов, чтобы отдельно взятые пользователи слишком сильно не перегружали  сервер. Подобное ограничение есть и у ВКонтакте, в документации которой указано, что можно делать не более 3-х запросов в секунду. 

Чтобы не следить за частотой отправки запросов с секундомером в руках, мы можем после каждого запроса делать паузу. В этом случае, даже если код будет выполняться на самом быстром компьютере, мы не нарушим установленное ограничение, т.к. периодичность отправки запросов будет искусственно замедлена. Воспользуемся библиотекой time и методом sleep (пауза в 0.5 секунды добавлена в конце цикла):

In [ ]:
import time  
  
count = 1000  
offset = 0  
user_ids = []  
while offset < 10000:  
    print('Выгружаю {} пользователей с offset = {}'.format(count, offset))     
    params = {  
        'group_id': 'vk',  
        'v': 5.95,  
        'count': count,  
        'offset': offset,  
        'access_token': token  
    }    
    # такой же запрос, как в прошлый раз  
    r = requests.get(url, params = params)  
    data = r.json()     
    user_ids += data['response']['items']    
  
    # увеличиваем смещение на количество строк выгрузки  
    offset += count  
    
    print('Ожидаю 0.5 секунды...')  
    time.sleep(0.5)  

## Лайки, репосты и комментарии
Ещё одна полезная вещь, которую можно получить,  —  это количество взаимодействий с постами через API новостной ленты. На данном этапе будем использовать самый простой вариант: берем последние 10 постов группы. Мы продолжаем работать с группой https://vk.com/vk.

Начнем с  формирования запроса к API ВКонтакте методом wall.get():

In [22]:
import requests  
from pprint import pprint  

In [23]:
url = 'https://api.vk.com/method/wall.get'  
token = '5d2f1be85d2f1be85d2f1be89d5d5901ae55d2f5d2f1be83d21d4ebe69afdbbac49afae'
params = {'domain': 'vk', 'filter': 'owner', 'count': 10,
          'offset': 0, 'access_token': token, 'v': 5.95}  

response = requests.get(url, params = params)  
response.json()

{'response': {'count': 429,
  'items': [{'id': 1148348,
    'from_id': -22822305,
    'owner_id': -22822305,
    'date': 1608903643,
    'marked_as_ads': 0,
    'post_type': 'post',
    'text': 'Благодаря форс-мажорному 2020-му все мы обзавелись онлайн-решениями на любые случаи жизни. ВКонтакте представила 220 крупных обновлений — чтобы вы легко справлялись с заботами и были на связи с теми, кто дорог.\n\nНапример, угадывали их желания и сразу заказывали подарок с доставкой до двери. Или созванивались всем курсом, чтобы поздравить любимого преподавателя с юбилеем — даже если у него нет страницы ВКонтакте. В этом году кто-то стал звездой в Клипах, а кто-то нашёл новых друзей, зажигая в чатах VK Fest. \n\nМы собрали самые громкие новинки и события ВКонтакте в насыщенном обзоре. Посмотрите, какие сервисы и функции вы ещё не попробовали: vk.com/blog/2020',
    'is_pinned': 1,
    'attachments': [{'type': 'link',
      'link': {'url': 'https://vk.com/blog/2020',
       'title': '12 мгновени

Видим, что сначала идёт общее количество постов, а по ключу 'items'  —  сами посты. Посмотрим на отдельный пост:

In [24]:
response.json()['response']['items'][0]  

{'id': 1148348,
 'from_id': -22822305,
 'owner_id': -22822305,
 'date': 1608903643,
 'marked_as_ads': 0,
 'post_type': 'post',
 'text': 'Благодаря форс-мажорному 2020-му все мы обзавелись онлайн-решениями на любые случаи жизни. ВКонтакте представила 220 крупных обновлений — чтобы вы легко справлялись с заботами и были на связи с теми, кто дорог.\n\nНапример, угадывали их желания и сразу заказывали подарок с доставкой до двери. Или созванивались всем курсом, чтобы поздравить любимого преподавателя с юбилеем — даже если у него нет страницы ВКонтакте. В этом году кто-то стал звездой в Клипах, а кто-то нашёл новых друзей, зажигая в чатах VK Fest. \n\nМы собрали самые громкие новинки и события ВКонтакте в насыщенном обзоре. Посмотрите, какие сервисы и функции вы ещё не попробовали: vk.com/blog/2020',
 'is_pinned': 1,
 'attachments': [{'type': 'link',
   'link': {'url': 'https://vk.com/blog/2020',
    'title': '12 мгновений 2020-го: главные новинки',
    'caption': 'vk.com',
    'description

Нужная нам статистика находится в полях 'comments', 'likes' и 'reposts'. Соберем итоговую статистику для каждого поста в словарь stats. В качестве ключа будем использовать начало статьи, в качестве значения — список с тремя интересующими нас метриками и временем публикации: [комментарии, лайки, репосты, дата публикации]

In [25]:
stats = {}  
              
for record in response.json()['response']['items'][:]:  
    title = record['text'][:30]  
    if title:  
        stats[title] = [record['comments']['count'], record['likes']['count'], record['reposts']['count'], record['date'] ]  

pprint(stats)  

{'Благодаря форс-мажорному 2020-': [5065, 1483, 341, 1608903643],
 'В наступающий год — со свежим ': [3184, 2896, 461, 1608033618],
 'В этом году у нас было много т': [250, 1580, 106, 1607943610],
 'Вот уже четвёртый год мы подде': [2750, 3000, 456, 1608897114],
 'Встречайте музыкальных куратор': [2096, 2511, 484, 1603796912],
 'Мы решили не ждать курантов и ': [870, 2527, 142, 1607689227],
 'Получать поздравления приятно,': [1376, 2872, 194, 1602506077],
 'Сообщества были и остаются одн': [641, 1917, 164, 1608105691],
 'Что поможет найти лучший автос': [1079, 5414, 4134, 1608548489]}


### Задание
Напишите функцию get_smm_index(group_name, token), которая по имени группы и авторизационному токену API возвращает smm_index группы - сумму лайков, комментариев и репостов для последних 10 постов, поделённую на количество участников в группе.

In [30]:
def get_smm_index(group_name, token):
    url_1 = 'https://api.vk.com/method/groups.getMembers'
    params_1 = {'group_id': group_name, 'v': 5.95, 'access_token': token}  
    response_1 = requests.get(url_1, params = params_1)  
    count_users = response_1.json()['response']['count'] 
    
    url_2 = 'https://api.vk.com/method/wall.get' 
    params_2 = {'domain': group_name, 'filter': 'owner', 'count': 11,
              'offset': 0, 'access_token': token, 'v': 5.95}  
    response_2 = requests.get(url_2, params = params_2)  
    
    comments = 0
    likes = 0
    reposts = 0
    for record in response_2.json()['response']['items'][:]:
        comments += record['comments']['count']
        likes += record['likes']['count']
        reposts += record['reposts']['count']
    
    smm_index = (comments + likes + reposts) / count_users
    return smm_index

In [31]:
token = '5d2f1be85d2f1be85d2f1be89d5d5901ae55d2f5d2f1be83d21d4ebe69afdbbac49afae'
get_smm_index('vk', token)

0.005493724749220394